In [236]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
import time

In [237]:
#workouts = pd.read_csv('workouts.csv')
#workouts = workouts[workouts.user_id!=2509]
#workouts = workouts[workouts.user_id!=2461]
#workouts.to_csv('workouts_temp.csv')

#Manually input missing days

workouts = pd.read_csv('workouts_temp.csv')
workouts.head()

,user_id,start,end,date_md,sport_id,name,intensity_score,z1,z2,z3,z4,z5
0,828,9/15/2015 20:08,9/15/2015 22:56,09/15/15,35,Track & Field,20.508564,1066,1810,1432,4668,679
1,828,NaN,NaN,09/16/15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,828,NaN,NaN,09/17/15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,828,NaN,NaN,09/18/15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,828,9/19/2015 12:46,9/19/2015 14:09,09/19/15,35,Track & Field,16.823285,506,601,1861,696,22


In [238]:
#Getting workouts down to 1 per day, with the summation of z1/z2/z3/z4/z5
workouts2 = workouts.groupby(['user_id', 'date_md']).sum().reset_index()
workouts2 = workouts2.sort(['user_id', 'date_md'], ascending=[True, True])
workouts2['z1'].fillna(0, inplace=True)
workouts2['z2'].fillna(0, inplace=True)
workouts2['z3'].fillna(0, inplace=True)
workouts2['z4'].fillna(0, inplace=True)
workouts2['z5'].fillna(0, inplace=True)
workouts2['intensity_score'].fillna(0, inplace=True)

In [239]:
workouts2.head()

,user_id,date_md,sport_id,intensity_score,z1,z2,z3,z4,z5
0,828,09/15/15,35,20.508564,1066,1810,1432,4668,679
1,828,09/16/15,NaN,0.000000,0,0,0,0,0
2,828,09/17/15,NaN,0.000000,0,0,0,0,0
3,828,09/18/15,NaN,0.000000,0,0,0,0,0
4,828,09/19/15,35,16.823285,506,601,1861,696,22


In [240]:
def epoch_end_convert(df):
    '''
    takes dataframe and converts date column to epoch milliseconds
    inputs:
        df = dataframe name
    '''
    return (int(time.mktime(time.strptime(df['date_md']+' 23:59', '%m/%d/%y %H:%M'))) - 4400)*1000

In [241]:
#workouts_subset['end_epoch'] = workouts_subset.apply(epoch_end_convert, axis=1)
workouts2['end_epoch'] = workouts2.apply(epoch_end_convert, axis=1)

In [242]:
workouts2.head()

,user_id,date_md,sport_id,intensity_score,z1,z2,z3,z4,z5,end_epoch
0,828,09/15/15,35,20.508564,1066,1810,1432,4668,679,1442371540000
1,828,09/16/15,NaN,0.000000,0,0,0,0,0,1442457940000
2,828,09/17/15,NaN,0.000000,0,0,0,0,0,1442544340000
3,828,09/18/15,NaN,0.000000,0,0,0,0,0,1442630740000
4,828,09/19/15,35,16.823285,506,601,1861,696,22,1442717140000


#Adding in Time Periods (For Either Bikes or Races)

In [243]:
date_markers = pd.read_csv('epoch_dates.csv')

In [244]:
date_markers.head()

,ET_Date,date_start_epoch,race_period,bike_period,race_period_start,race_period_end,bike_period_start,bike_period_end
0,9/15/2015 0:00,1442289600000,1,1,1,0,1,0
1,9/16/2015 0:00,1442376000000,1,1,0,0,0,0
2,9/17/2015 0:00,1442462400000,1,1,0,0,0,0
3,9/18/2015 0:00,1442548800000,1,1,0,0,0,0
4,9/19/2015 0:00,1442635200000,1,1,0,0,0,0


In [245]:
date_markers = date_markers[date_markers['race_period_start'] > 0].reset_index()
#dictionary of race period to start epoch
race_start_dict = {}
for i in range(0, len(date_markers)):
    race_start_dict[date_markers.values[i][3]] = date_markers.values[i][2]

#date_markers = date_markers[date_markers['bike_period_start'] > 0].reset_index()
#bike_start_dict = {}
#for i in range(0, len(date_markers)):
#    bike_start_dict[date_markers.values[i][4]] = date_markers.values[i][2]


In [246]:
users = list(set(workouts2.user_id.values))

In [247]:
race_start_dict

{1L: 1442289600000L,
 2L: 1443240000000L,
 3L: 1445054400000L,
 4L: 1446264000000L,
 5L: 1447473600000L}

In [248]:
def add_race(df):
    if (df.end_epoch >= race_start_dict[1]) and (df.end_epoch < race_start_dict[2]):
        return 1
    elif (df.end_epoch >= race_start_dict[2]) and (df.end_epoch < race_start_dict[3]):
        return 2
    elif (df.end_epoch >= race_start_dict[3]) and (df.end_epoch < race_start_dict[4]):
        return 3
    elif (df.end_epoch >= race_start_dict[4]) and (df.end_epoch < race_start_dict[5]):
        return 4
    elif (df.end_epoch >= race_start_dict[5]) and (df.end_epoch < 1448164800000):
        return 5
    else:
        return 0


In [249]:
def add_bike(df, bike_start_dict):
    for user in users:
        if df.user_id == user:
            if (df.end_epoch <= bike_start_dict[user, 1]):
                return 1
            elif (df.end_epoch <= bike_start_dict[user, 2]) and (df.end_epoch >= bike_start_dict[user, 1]):
                return 2
            elif (df.end_epoch <= bike_start_dict[user, 3]) and (df.end_epoch >= bike_start_dict[user, 2]):
                return 3
            elif (df.end_epoch <= bike_start_dict[user, 4]) and (df.end_epoch >= bike_start_dict[user, 3]):
                return 4
            elif (df.end_epoch <= bike_start_dict[user, 5]) and (df.end_epoch >= bike_start_dict[user, 4]):
                return 5
            elif (df.end_epoch <= bike_start_dict[user, 6]) and (df.end_epoch >= bike_start_dict[user, 5]):
                return 6
            elif (df.end_epoch <= bike_start_dict[user, 7]) and (df.end_epoch >= bike_start_dict[user, 6]):
                return 7
            elif (df.end_epoch <= bike_start_dict[user, 8]) and (df.end_epoch >= bike_start_dict[user, 7]):
                return 8
            elif (df.end_epoch >= bike_start_dict[user, 8]):
                return 9
                

In [250]:
def add_periods(date_markers, df, buildup_days=0, version=None):
    if version==None:
        print "Enter Either 'race' or 'bike'!"
    if version=="race":
        #Collapse the data frame into only the start and end dates for each race period
        date_markers = date_markers[date_markers['race_period_start'] > 0].reset_index()
        #dictionary of race period to start epoch
        race_start_dict = {}
        for i in range(0, len(date_markers)):
            race_start_dict[date_markers.values[i][3]] = date_markers.values[i][2]
        #Now label each race period as 1, 2, 3, 4, or 5
        df['race_period'] = df.apply(add_race, axis=1)
        
        #add in buildup days
        df=df.sort(['user_id', 'race_period', 'end_epoch'], ascending=[True, True, False]).reset_index(drop=True)
        race_groups = df.groupby(['user_id', 'race_period']).cumcount()
        build_days = pd.DataFrame(dict(buildup_days = race_groups))
        new_df = df.join(build_days, how="left")
        return_df = new_df
    
    elif version=="bike":
        #same process, but with bike efforts
        bike_dates = pd.read_csv('huxc_bike_distances.csv')
        bike_dates['next_day'] = bike_dates['date_start_epoch'] + 86400000
        bike_start_dict={} #key is the user_id and bike period, value is the date start epoch
        for i in range(0, len(bike_dates)):
            bike_start_dict[bike_dates.values[i][0], bike_dates.values[i][1]] = bike_dates.values[i][5]
        #cycle through bike_start dict, enter placement values so day gets filled
        for user in users:
            for i in range(1, 10):
                if np.isnan(bike_start_dict[user, i]) and i == 1:
                    bike_start_dict[user, i] = bike_start_dict[2469, i]
                if np.isnan(bike_start_dict[user, i]) and i == 2:
                    bike_start_dict[user, i] = bike_start_dict[2469, i]
                if np.isnan(bike_start_dict[user, i]) and i == 3:
                    bike_start_dict[user, i] = bike_start_dict[2469, i] 
                if np.isnan(bike_start_dict[user, i]) and i == 4:
                    bike_start_dict[user, i] = bike_start_dict[2469, i]
                if np.isnan(bike_start_dict[user, i]) and i == 5:
                    bike_start_dict[user, i] = bike_start_dict[2469, i]
                if np.isnan(bike_start_dict[user, i]) and i == 6:
                    bike_start_dict[user, i] = bike_start_dict[2469, i]
                if np.isnan(bike_start_dict[user, i]) and i == 7:
                    bike_start_dict[user, i] = bike_start_dict[2469, i]
                if np.isnan(bike_start_dict[user, i]) and i == 8:
                    bike_start_dict[user, i] = bike_start_dict[2469, i]
                if np.isnan(bike_start_dict[user, i]) and i == 9:
                    bike_start_dict[user, i] = bike_start_dict[2469, i]
        df['bike_period'] = df.apply(add_bike, axis=1, args=(bike_start_dict,))
        df=df.sort(['user_id', 'bike_period', 'end_epoch'], ascending=[True, True, False]).reset_index(drop=True)
        bike_groups = df.groupby(['user_id', 'bike_period']).cumcount()
        build_days = pd.DataFrame(dict(buildup_days = bike_groups))
        new_df = df.join(build_days, how="left")
        return_df = new_df
        
    if buildup_days != 0:
        return_df = return_df[return_df['buildup_days'] < buildup_days]
    
    final_return_df = return_df[return_df['race_period']!=0]
    return return_df

In [251]:
test = add_periods(date_markers=date_markers, df=workouts2, buildup_days=14, version="race")
test = test.sort(['user_id', 'date_md'], ascending=['True', 'True']).reset_index(drop=True)
test = test[test['race_period']!=0]
test = test.drop(['sport_id'], axis=1)

#Aggregating Weighted Rolling Average of Variables Leading up To Races/Bikes

In [252]:
def filter_nans(x, y):
    filtered = filter(lambda o: not np.isnan(o[0]) and not np.isnan(o[1]), zip(x, y))    
    return [el[0] for el in filtered], [el[1] for el in filtered]

In [253]:
def get_item(df_column_tuple):
    return df_column_tuple[0], df_column_tuple[1]

In [254]:
def weighted_rolling_average(data):
    #must take into account nan
    #number of things in values
    n = len(data)
    weights = []
    values = []
    for i in range(0, n):
        values.append(data[i])
        #don't care about race day
        weights.append(i+1)
    numbers = [w*v for w,v in zip(weights,values)]
    filtered_numbers, filtered_weights = filter_nans(numbers, weights)
    #print filtered_numbers, filtered_weights
    return sum(filtered_numbers)/sum(filtered_weights)

In [255]:
def weighted_rolling_average_df(df, activity_type, variable):
    weighted_avg_dict = {}
    periods = []
    if activity_type=='bike':
        periods = [1,2,3,4,5,6,7,8,9]
        for u in users:
            user_df = df[df['user_id']==u]
            for i in range(0, len(periods)):
                user_period_df=user_df[user_df['bike_period']==periods[i]]
                w_r_a = weighted_rolling_average(user_period_df[variable].reset_index(drop=True))
                weighted_avg_dict[u, periods[i], variable] = w_r_a
        
    elif activity_type=='race':
        periods = [1,2,3,4,5]
        for u in users:
            user_df = df[df['user_id']==u]
            for i in range(0, len(periods)):
                user_period_df=user_df[user_df['race_period']==periods[i]]
                w_r_a = weighted_rolling_average(user_period_df[variable].reset_index(drop=True))
                weighted_avg_dict[u, periods[i], variable] = w_r_a
                
    
    agg_df = pd.DataFrame(weighted_avg_dict.items(), columns=['temp', variable+'_temp'])
    agg_df['user_id'], agg_df[activity_type+'_period'] = zip(*agg_df['temp'].map(get_item))
    agg_df[variable] = agg_df[variable+'_temp']
    agg_df = agg_df.sort(['user_id', activity_type+'_period'], ascending = [True, True]).reset_index(drop=True)
    agg_df = agg_df.drop(['temp', variable+'_temp'], axis=1)
    return agg_df

In [256]:
new_df = weighted_rolling_average_df(test, 'race', 'intensity_score')
new_df2 = weighted_rolling_average_df(test, 'race', 'z1')
new_df3 = weighted_rolling_average_df(test, 'race', 'z2')
new_df4 = weighted_rolling_average_df(test, 'race', 'z3')
new_df5 = weighted_rolling_average_df(test, 'race', 'z4')
new_df6 = weighted_rolling_average_df(test, 'race', 'z5')

merge_df1 = pd.merge(new_df, new_df2, how='left')
merge_df2 = pd.merge(merge_df1, new_df3, how='left')
merge_df3 = pd.merge(merge_df2, new_df4, how='left')
merge_df4 = pd.merge(merge_df3, new_df5, how='left')
merged_df = pd.merge(merge_df4, new_df6, how='left')

dummies = pd.get_dummies(merged_df['user_id'], prefix='user')
final_df = pd.concat([merged_df, dummies], axis=1)
final_df = final_df.drop(['user_828'], axis=1)
race_times = pd.read_csv('huxc_race_times.csv')
race_times=race_times[race_times['user_id']!=2461]
race_times=race_times[race_times['user_id']!=2509]
final_race_df = final_df.merge(race_times, how='left')
final_race_df = final_race_df[final_race_df['user_id']!=2439]
final_race_df.to_csv('workouts_final_race_df.csv')

##Same Process, but for bike efforts

In [257]:
test2 = add_periods(date_markers=date_markers, df=workouts2, buildup_days=8, version="bike")
test2 = test2.sort(['user_id', 'date_md'], ascending=['True', 'True']).reset_index(drop=True)
test2 = test2.drop(['sport_id', 'race_period'], axis=1)
test2.head(10)

,user_id,date_md,intensity_score,z1,z2,z3,z4,z5,end_epoch,bike_period,buildup_days
0,828,09/21/15,23.455466,663,1927,1377,2781,28,1442889940000,1,7
1,828,09/22/15,18.825382,767,992,1291,806,1323,1442976340000,1,6
2,828,09/23/15,20.871214,268,512,703,1452,89,1443062740000,1,5
3,828,09/24/15,0.000000,0,0,0,0,0,1443149140000,1,4
4,828,09/25/15,19.797324,1453,1601,1803,2406,630,1443235540000,1,3
5,828,09/26/15,18.177086,538,353,557,2124,365,1443321940000,1,2
6,828,09/27/15,15.833133,423,281,133,1604,1,1443408340000,1,1
7,828,09/28/15,23.660118,729,864,2118,2543,67,1443494740000,1,0
8,828,09/29/15,19.757302,458,1020,1510,3902,35,1443581140000,2,6
9,828,09/30/15,29.896431,399,857,1081,2399,4,1443667540000,2,5


In [258]:
new_df = weighted_rolling_average_df(test2, 'bike', 'intensity_score')
new_df2 = weighted_rolling_average_df(test2, 'bike', 'z1')
new_df3 = weighted_rolling_average_df(test2, 'bike', 'z2')
new_df4 = weighted_rolling_average_df(test2, 'bike', 'z3')
new_df5 = weighted_rolling_average_df(test2, 'bike', 'z4')
new_df6 = weighted_rolling_average_df(test2, 'bike', 'z5')

merge_df1 = pd.merge(new_df, new_df2, how='left')
merge_df2 = pd.merge(merge_df1, new_df3, how='left')
merge_df3 = pd.merge(merge_df2, new_df4, how='left')
merge_df4 = pd.merge(merge_df3, new_df5, how='left')
merged_df = pd.merge(merge_df4, new_df6, how='left')

dummies = pd.get_dummies(merged_df['user_id'], prefix='user')
final_df = pd.concat([merged_df, dummies], axis=1)
final_df = final_df.drop(['user_828'], axis=1)
bike_results = pd.read_csv('huxc_bike_distances.csv')
bike_results = bike_results[bike_results['user_id']!=2461]
bike_results = bike_results[bike_results['user_id']!=2509]
bike_results = bike_results.drop(['date_md', 'date_start_epoch'], axis=1)
final_race_df = final_df.merge(bike_results, how='left')
final_race_df = final_race_df[final_race_df['user_id']!=2439]
final_race_df.to_csv('workouts_final_bike_df.csv')